In [1]:
%pip install -q -U google-generativeai
%pip install youtube_transcript_api
%pip install PyTube openai moviepy
import json
import subprocess
import shutil
import os

import google.generativeai as genai

from openai import OpenAI

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


c:\Users\Anand Bachker\Desktop\pipeline\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

OPENAI_KEY = userdata.get('OPENAI_KEY')
client = OpenAI(
    api_key=OPENAI_KEY
)

for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)


models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-flash
models/gemini-1.5-flash-001
models/gemini-1.5-flash-latest
models/gemini-1.5-pro
models/gemini-1.5-pro-001
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


In [3]:
from youtube_transcript_api import YouTubeTranscriptApi
from pytube import YouTube
import moviepy.editor as mp
import re

def get_video_id(url):
    # Extract video id from YouTube URL
    video_id = re.search(r'(?<=v=)[^&#]+', url)
    if video_id is None:
        video_id = re.search(r'(?<=be/)[^&#]+', url)
    if video_id:
        return video_id.group(0)
    else:
        raise ValueError("Invalid YouTube URL")

def get_raw_transcript(url):
    video_id = get_video_id(url)
    transcript_list = YouTubeTranscriptApi.get_transcript(video_id)
    return transcript_list


def download_and_trim(url, base_directory, content_list):
    try:
        
        # If the directory already exists, remove it first
        if os.path.exists(base_directory):
            shutil.rmtree(base_directory)

        yt = YouTube(url)
        video = yt.streams.first().download(output_path=base_directory)
        print(f"Downloaded video: {video}")


        # Ensure the clips directory exists
        clips_directory = os.path.join(base_directory, "clips")
        os.makedirs(clips_directory, exist_ok=True)

        for content in content_list:
            start_time = content["start_time"]
            end_time = content["end_time"]
            title = content["title"]
            output_file = os.path.join(clips_directory, f"{title}.mp4")

            # Construct the ffmpeg command
            ffmpeg_command = [
                "ffmpeg",
                "-i", video,
                "-ss", str(start_time),
                "-to", str(end_time),
                "-c", "copy",
                output_file
            ]

            # Run the ffmpeg command
            subprocess.run(ffmpeg_command, check=True)

    except Exception as e:
        print(f"An error occurred: {e}")

def delete_folder(dir):
  shutil.rmtree(dir)



In [4]:
base_prompt = "Extract a list of all {topic_prompt} and the corresponding start and end timestamps where they are discussed in the following YouTube transcript. Ensure duration is within 20 to 80 seconds with recommended length of 30 sec,it is IMPORTANT video length should not be less than 20 sec and should be bound by limits. Format the output as a JSON array:"

output_format = """

[
    {
      "title": "string",
      "info": "string",
      "why_clip_was_chosen": "string",
      "url":string,
      "start_time": int,
      "end_time": int
    },
    ...
]


"""

additional_base_prompt = """
if nothing can be extracted return an empty array i.e. []

{url_source_prompt}

dont return anything else as output it should always be an array

Transcript:

"""


def get_message(transcript, topic_prompt="Books", url_source=""):
  url_source_prompt = ""
  if url_source:
    url_source_prompt = f"\nFor the URL, use the {url_source} link relevant to the discussed topic."
  else:
    url_source_prompt = "keep the 'url' filed empty don't put any value in that key"


  base_prompt_with_topic = base_prompt.format(topic_prompt=topic_prompt)
  additional_base_prompt_with_url_source_prompt = additional_base_prompt.format(url_source_prompt=url_source_prompt)

  transcript_text = "\n".join([f"{item['start']} - {item['text']}" for item in transcript])
  result = base_prompt_with_topic + output_format + additional_base_prompt_with_url_source_prompt + transcript_text
  return result

In [5]:
def get_llm_output(messageText, service, model):
  if service == "gemini":
      model = genai.GenerativeModel(model)
      output = model.generate_content(messageText)
      return output.text
  elif service == "openai":
      chat_completion = client.chat.completions.create(
        messages=[
          {
            "role": "user",
            "content": messageText,
          }
        ],
        model=model,
      )
      return chat_completion.choices[0].message.content
  else:
    return "[]"

In [6]:
def get_clips(url, topic_prompt, service = "gemini", model="gemini-pro", url_source = ""):
  transcript = get_raw_transcript(url)
  message = get_message(transcript, topic_prompt, url_source)
  llmOutput = get_llm_output(message ,service,model)
  llmOutput = llmOutput.replace("```", "")
  llmOutput = llmOutput.replace("json", "")
  print(llmOutput)
  result = json.loads(llmOutput)
  return result


In [7]:
url = "https://www.youtube.com/watch?v=HMX0vrJj5nM"
extracted_clips = get_clips(
    url = url,
    topic_prompt = "gadgets mentioned",
    service = "gemini",
    model = "gemini-pro",
    url_source = ""
  )

[
  {
    "title": "Smart Trash Can",
    "info": "Streamlined way to dispose of trash, with hands-free operation and separate compartments for recycling.",
    "why_clip_was_chosen": null,
    "url": null,
    "start_time": 12.179,
    "end_time": 30.84
  },
  {
    "title": "Ember Mug",
    "info": "Temperature-controlled mug that keeps hot beverages at the ideal drinking temperature, promoting productivity and reducing caffeine intake.",
    "why_clip_was_chosen": null,
    "url": null,
    "start_time": 62.399,
    "end_time": 89.34
  },
  {
    "title": "Electric Toothbrush with LED Display",
    "info": "Advanced toothbrush with an LED display and multiple cleaning modes, providing a dentist-clean feeling and promoting oral health.",
    "why_clip_was_chosen": null,
    "url": null,
    "start_time": 119.46,
    "end_time": 157.92
  },
  {
    "title": "Balmuda Toaster",
    "info": "Premium toaster that uses steam to create crispy exteriors and fluffy interiors, offering an elev

In [8]:
print("Total items found = ",len(extracted_clips))
for clip in extracted_clips:
  delta = round(clip['end_time']-clip['start_time'],3)
  print(clip['title'],delta)

Total items found =  11
Smart Trash Can 18.661
Ember Mug 26.941
Electric Toothbrush with LED Display 38.46
Balmuda Toaster 40.681
Robo Rock S7 Vacuum Cleaner 39.48
AirPods Pro 2 65.4
Theroface 39.06
Air Up Steel Bottle 32.94
iPhone 14 Pro Max 27.24
Google Pixel 7 Pro 12.3
16-inch MacBook with M1 Max Chip 34.26


In [9]:
base_directory = "./downloads/io"

download_and_trim(
    url = url,
    base_directory = base_directory,
    content_list = extracted_clips
  )


Downloaded video: c:\Users\Anand Bachker\Desktop\pipeline\./downloads/io\Ive tested 5000 gadgets - These are the 10 I actually Use.mp4
